In [ ]:
import pandas as pd
import os


: 

In [ ]:
# 1. 파일 경로 설정
current_folder = os.path.dirname(os.path.abspath(__file__))
# 아까 저장한 '커피' 데이터 파일 이름이 맞는지 꼭 확인!
file_name = "seoul_coffee_numeric.csv" 
file_path = os.path.join(current_folder, file_name)

# 2. 데이터 불러오기 (판다스가 알아서 숫자로 가져옴!)
df = pd.read_csv(file_path)
print(f"📂 데이터 로드 완료! (총 {len(df)}개)")

# -------------------------------------------------------------
# 3. 분기(1, 2, 3, 4) 정보 추출
# -------------------------------------------------------------
# STDR_YYQU_CD (예: 20231)에서 맨 끝자리만 잘라내기
# 이건 숫자니까 문자로 잠깐 바꿨다가(.astype(str)) 잘라야 해
df['QUARTER_NUM'] = df['STDR_YYQU_CD'].astype(str).str[-1]

print("⚙️ 분기 구분 완료!")

# -------------------------------------------------------------
# 4. [계산] 상권별 분기 평균 & 총 평균
# -------------------------------------------------------------
# 기준: 상권_코드(TRDAR_CD) 별로 묶어서 계산

# (A) 분기별 평균 (1~4분기 각각 얼마?)
quarterly_avg = df.pivot_table(
    index='TRDAR_CD', 
    columns='QUARTER_NUM', 
    values='THSMON_SELNG_AMT', 
    aggfunc='mean'
)
# 컬럼 이름 변경 (1 -> 1분기_평균)
quarterly_avg.columns = [f'{c}분기_평균' for c in quarterly_avg.columns]

# (B) 3개년 전체 평균
total_avg = df.groupby('TRDAR_CD')['THSMON_SELNG_AMT'].mean().rename('3개년_총평균')

# -------------------------------------------------------------
# 5. [병합] 원본 데이터에 계산 결과 붙이기
# -------------------------------------------------------------
# 상권 코드(TRDAR_CD)가 같은 줄에다가 값을 붙여넣기
df = df.merge(quarterly_avg, on='TRDAR_CD', how='left')
df = df.merge(total_avg, on='TRDAR_CD', how='left')

# -------------------------------------------------------------
# 6. 컬럼 위치 예쁘게 정렬하기
# -------------------------------------------------------------
# 'THSMON_SELNG_AMT'(매출액) 옆에다가 만든 컬럼들을 쏙 끼워넣자

cols = df.columns.tolist()
new_cols = ['1분기_평균', '2분기_평균', '3분기_평균', '4분기_평균', '3개년_총평균']

# (1) 리스트 맨 뒤에 붙은 새 컬럼들을 잠깐 뺌
for col in new_cols:
    if col in cols: cols.remove(col)

# (2) 매출액 컬럼 위치 찾기
target_idx = cols.index('THSMON_SELNG_AMT')

# (3) 그 바로 뒤(+1)에 하나씩 삽입
for i, col in enumerate(new_cols):
    cols.insert(target_idx + 1 + i, col)

# (4) 순서 적용
df = df[cols]

# -------------------------------------------------------------
# 7. 저장
# -------------------------------------------------------------
save_name = "seoul_coffee_analysis.csv"
save_path = os.path.join(current_folder, save_name)

df.to_csv(save_path, index=False, encoding="utf-8-sig")

print("\n" + "="*50)
print("✨ 분석 완료! 매출 비교표를 만들었어.")
print(f"💾 저장된 파일: {save_name}")
# 확인용 출력
print(df[['TRDAR_SE_CD_NM', 'THSMON_SELNG_AMT', '3개년_총평균']].head())